# Clayton Copula model
用來計算兩組數據的左尾相關性

In [5]:
from scipy.optimize import minimize
import scipy.stats as stats
import yfinance as yf
import numpy as np

# Clayton Copula 模型
def clayton_log_likelihood(theta, u, v):
    if theta <= 0:
        return np.inf
    n = len(u)
    term1 = (theta + 1) * np.sum(np.log(u * v))
    term2 = np.sum((u ** (-theta) + v ** (-theta) - 1) ** (-1 / theta - 1) * (u ** (-theta - 1) * v ** (-theta - 1)))
    log_likelihood = n * np.log(theta + 1) - term1 - term2
    return -log_likelihood

def eval_clayton_theta(data1, data2):
    # 估計邊際分布
    # 假設收益率服從正態分布
    params1 = stats.norm.fit(data1)
    params2 = stats.norm.fit(data2)

    # 轉換為均勻分布
    u1 = stats.norm.cdf(data1, *params1)
    u2 = stats.norm.cdf(data2, *params2)


    # 嘗試不同初始值
    initial_guesses = [0.1, 0.5, 1, 2, 5]
    best_theta = None
    best_log_likelihood = np.inf

    for initial_guess in initial_guesses:
        result = minimize(clayton_log_likelihood, x0=initial_guess, args=(u1, u2), bounds=[(0.0001, None)])
        if result.fun < best_log_likelihood:
            best_log_likelihood = result.fun
            best_theta = result.x[0]

    # 計算Kendall's tau
    tau = best_theta / (best_theta + 2)

    ### theta為coupla參數，數值越大代表越相關
    print(f'Estimated theta: {best_theta}')

## 使用範例
分數 > 0 代表有關係，分數越高代表關聯性越強

In [6]:

# 下載股票資料
data1 = yf.download('2330.TW', start='2020-01-01', end='2023-01-01')['Adj Close'].pct_change().dropna()
data2 = yf.download('2454.TW', start='2020-01-01', end='2023-01-01')['Adj Close'].pct_change().dropna()

eval_clayton_theta(data1, data2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Estimated theta: 0.10592974037562221
